#### ID 2040

```Summarize the number of customers and transactions for each month in 2017, keeping transactions that were greater or equal to $5.```

In [ ]:
%%sql
WITH cte AS (SELECT transaction_id, SUM(sales)
             FROM wfm_transactions
             WHERE EXTRACT(YEAR FROM transaction_date) = 2017
             GROUP BY transaction_id
             HAVING SUM(sales) >= 5)
SELECT EXTRACT(MONTH FROM transaction_date) AS month,
       COUNT(DISTINCT customer_id)          AS customers,
       COUNT(DISTINCT transaction_id)       AS transactions
FROM cte
         JOIN wfm_transactions USING (transaction_id)
GROUP BY month
ORDER BY month

In [ ]:
df = wfm_transactions
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
transactions_list = df.query('year == 2017').groupby('transaction_id', as_index=False).agg(total_sales=('sales', 'sum')).query('total_sales >= 5')['transaction_id'].to_list()
df.query('transaction_id.isin(@transactions_list)').groupby('month', as_index=False).agg(customers=('customer_id', 'nunique'), transactions=('transaction_id', 'nunique'))

#### ID 2041

```You work for a multinational company that wants to calculate total sales across all their countries they do business in. You have 2 tables, one is a record of sales for all countries and currencies the company deals with, and the other holds currency exchange rate information. Calculate the total sales, per quarter, for the first 2 quarters in 2020, and report the sales in USD currency.```

In [ ]:
%%sql
SELECT EXTRACT(QUARTER FROM date)        AS quarter,
       SUM(sales_amount * exchange_rate) AS total_sales
FROM sf_exchange_rate er
         JOIN sf_sales_amount sa
              ON er.date = sa.sales_date AND sa.currency = er.source_currency
WHERE EXTRACT(QUARTER FROM date) IN (1, 2)
GROUP BY quarter

In [ ]:
df = pd.merge(sf_sales_amount, sf_exchange_rate, how='inner', right_on=['date', 'source_currency'], left_on=['sales_date', 'currency'])

df['amount_sales'] = df['sales_amount'] * df['exchange_rate']
df['year'] = df['sales_date'].dt.year
df['quarter'] = df['sales_date'].dt.quarter

df.query('year == 2020 & quarter.isin([1, 2])').groupby('quarter', as_index=False).agg(total_sales=('amount_sales', 'sum'))

#### ID 2043

```Return all employees who have never had an annual review. Your output should include the employee's first name, last name, hiring date, and termination date. List the most recently hired employees first.```

In [ ]:
%%sql
SELECT first_name,
       last_name,
       hire_date,
       termination_date
FROM uber_employees
WHERE id NOT IN (SELECT emp_id FROM uber_annual_review)
ORDER BY hire_date DESC

In [ ]:
employees_list = uber_annual_review['emp_id'].to_list()
df = uber_employees
df.query('~id.isin(@employees_list)')[['first_name', 'last_name', 'hire_date', 'termination_date']].sort_values('hire_date', ascending=False)